In [1]:
!pip install transformers

     |████████████████████████████████| 3.5 MB 5.2 MB/s 
     |████████████████████████████████| 596 kB 50.7 MB/s 
     |████████████████████████████████| 895 kB 40.9 MB/s 
     |████████████████████████████████| 6.8 MB 45.3 MB/s 
     |████████████████████████████████| 67 kB 4.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 6395311469943380946
 xla_global_id: -1]

In [1]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import tensorflow as tf
from transformers import BertTokenizer

In [3]:
df = pd.read_csv('merged.csv')
df.head()

,text,label
0,brooklyn 99 gambling,2
1,instagram ochish,4
2,douche nugget,0
3,permainan yang cuma 1 mb,0
4,#facebook2018,4


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40882 entries, 0 to 40881
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    40882 non-null  object
 1   label   40882 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 638.9+ KB


### **Data Preparation**

In [5]:
df.head()

,text,label
0,brooklyn 99 gambling,2
1,instagram ochish,4
2,douche nugget,0
3,permainan yang cuma 1 mb,0
4,#facebook2018,4


In [6]:
df['label'] = df['label'].astype(int)

In [7]:
print(df['label'].value_counts())

'''
The category labels are:

0 - adult
1 - drugs
2 - gambling
3 - violence
4 - entertainment
'''

4    15085
0    12139
1     6050
2     4640
3     2968
Name: label, dtype: int64


'\nThe category labels are:\n\n0 - adult\n1 - drugs\n2 - gambling\n3 - violence\n4 - entertainment\n'

In [8]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [9]:
token = tokenizer.encode_plus(
    df['text'].iloc[0], 
    max_length=256, 
    truncation=True, 
    padding='max_length', 
    add_special_tokens=True,
    return_tensors='tf'
)

In [10]:
# sample data
sample = ["this is a bert model tutorial", "we will fine-tune a bert model"]

# encode text
sent_id = tokenizer.batch_encode_plus(sample, padding=True, return_token_type_ids=False
    )

In [11]:
sent_id

{'input_ids': [[101, 1142, 1110, 170, 1129, 3740, 2235, 17463, 2916, 102, 0], [101, 1195, 1209, 2503, 118, 9253, 170, 1129, 3740, 2235, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [12]:
token.input_ids

<tf.Tensor: shape=(1, 256), dtype=int32, numpy=
array([[  101,  9304,  9753,  8068,  4850, 13376,   102,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0, 

In [13]:
X_input_ids = np.zeros((len(df), 256))
X_attn_masks = np.zeros((len(df), 256))

In [14]:
def generate_training_data(df, ids, masks, tokenizer):
    for i, text in tqdm(enumerate(df['text'])):
        tokenized_text = tokenizer.encode_plus(
            text,
            max_length=256, 
            truncation=True, 
            padding='max_length', 
            add_special_tokens=True,
            return_tensors='tf'
        )
        ids[i, :] = tokenized_text.input_ids
        masks[i, :] = tokenized_text.attention_mask
    return ids, masks

In [15]:
X_input_ids, X_attn_masks = generate_training_data(df, X_input_ids, X_attn_masks, tokenizer)

40882it [00:08, 4674.02it/s]


In [16]:
labels = np.zeros((len(df), 5))
labels.shape

(40882, 5)

In [17]:
labels[np.arange(len(df)), df['label'].values] = 1 # one-hot encoded target tensor

In [18]:
labels

array([[0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0.],
       ...,
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.]])

In [19]:
# creating a data pipeline using tensorflow dataset utility, creates batches of data for easy loading...
dataset = tf.data.Dataset.from_tensor_slices((X_input_ids, X_attn_masks, labels))
dataset.take(1) # one sample data

<TakeDataset element_spec=(TensorSpec(shape=(256,), dtype=tf.float64, name=None), TensorSpec(shape=(256,), dtype=tf.float64, name=None), TensorSpec(shape=(5,), dtype=tf.float64, name=None))>

In [20]:
def queryclassifiersetMapFunction(input_ids, attn_masks, labels):
    return {
        'input_ids': input_ids,
        'attention_mask': attn_masks
    }, labels

In [21]:
dataset = dataset.map(queryclassifiersetMapFunction) # converting to required format for tensorflow dataset 

In [22]:
dataset.take(1)

<TakeDataset element_spec=({'input_ids': TensorSpec(shape=(256,), dtype=tf.float64, name=None), 'attention_mask': TensorSpec(shape=(256,), dtype=tf.float64, name=None)}, TensorSpec(shape=(5,), dtype=tf.float64, name=None))>

In [23]:
dataset = dataset.shuffle(10000).batch(16, drop_remainder=True) # batch size, drop any left out tensor

In [24]:
dataset.take(1)

<TakeDataset element_spec=({'input_ids': TensorSpec(shape=(16, 256), dtype=tf.float64, name=None), 'attention_mask': TensorSpec(shape=(16, 256), dtype=tf.float64, name=None)}, TensorSpec(shape=(16, 5), dtype=tf.float64, name=None))>

In [25]:
p = 0.8
train_size = int((len(df)//16)*p) # for each 16 batch of data we will have len(df)//16 samples, take 80% of that for train.

In [26]:
train_size

2044

In [27]:
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)

### **Model**

In [28]:
from transformers import TFBertModel

In [29]:
model = TFBertModel.from_pretrained('bert-base-cased') # bert base model with pretrained weights

Downloading: 100%|██████████| 502M/502M [00:35<00:00, 14.9MB/s] 
Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [30]:
# defining 2 input layers for input_ids and attn_masks
input_ids = tf.keras.layers.Input(shape=(256,), name='input_ids', dtype='int32')
attn_masks = tf.keras.layers.Input(shape=(256,), name='attention_mask', dtype='int32')

bert_embds = model.bert(input_ids, attention_mask=attn_masks)[1] # 0 -> activation layer (3D), 1 -> pooled output layer (2D)
intermediate_layer = tf.keras.layers.Dense(512, activation='relu', name='intermediate_layer')(bert_embds)
output_layer = tf.keras.layers.Dense(5, activation='softmax', name='output_layer')(intermediate_layer) # softmax -> calcs probs of classes

query_classifier_model = tf.keras.Model(inputs=[input_ids, attn_masks], outputs=output_layer)
query_classifier_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                           

In [32]:
optim = tf.keras.optimizers.Adam(learning_rate=1e-5, decay=1e-6)
loss_func = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

In [33]:
query_classifier_model.compile(optimizer=optim, loss=loss_func, metrics=[acc])

In [1]:
hist = query_classifier_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=2
)

In [45]:
query_classifier_model.save('query_classifier_model')

INFO:tensorflow:Assets written to: query_classifier_model/assets


INFO:tensorflow:Assets written to: query_classifier_model/assets


In [ ]:
query_classifier_model

### **Prediction**

In [37]:
query_classifier_model = tf.keras.models.load_model('query_classifier_model')

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

def prepare_data(input_text, tokenizer):
    token = tokenizer.encode_plus(
        input_text,
        max_length=256, 
        truncation=True, 
        padding='max_length', 
        add_special_tokens=True,
        return_tensors='tf'
    )
    return {
        'input_ids': tf.cast(token.input_ids, tf.float64),
        'attention_mask': tf.cast(token.attention_mask, tf.float64)
    }

def make_prediction(model, processed_data, classes=['Adult Content', 'Drugs related content', 'Gambling related content', 'Violence Related content', 'Entertainment Related content']):
    probs = model.predict(processed_data)[0]
    return classes[np.argmax(probs)]

In [58]:
input_text = input('Enter search query here: ')
processed_data = prepare_data(input_text, tokenizer)
result = make_prediction(query_classifier_model, processed_data=processed_data)
print(f"Predicted Category: {result}")

Enter search query here: bomb 
Predicted Category: Adult Content
